In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.layers import Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers

import numpy as np
# 내가 가진 이미지를 학습시키기
# 학습셋의 변형을 설정하는 부분
train_datagen = ImageDataGenerator(rescale=1./255,# 주어진 이미지의 크기를 설정
                                  horizontal_flip = True,#수평 대칭 이미지를 50% 확률로 만듦
                                  width_shift_range= 0.1, # 전체 크기의 15% 범위에서 좌우로 이동
                                  height_shift_range=0.1, # 전체 크기의 15% 범위에서위,아래로 이동
                                  #rotation_range = 5, #정해진 각도만큼 회전
                                  shear_range=0.7, #좌표 하나를 고정시키고 나머지를 이동시킴
                                  #zoom_range=1.2, # 확대 또는 축소
                                  #vertical_flip=True, # 수직 대칭 이미지를 만듦 
                                  fill_mode='nearest' # 빈 공간을 채우는 방법
                                   # nearest : 비슷한 색으로
                                  )
train_generator = train_datagen.flow_from_directory(
    './data/train',# 학습셋이 있는 폴더의 위치
    target_size = (150, 150),
    batch_size = 5,
    class_mode = 'binary'
)
# 테스트셋 설정
# 이미지 부풀리기 과정을 진행하지 않음 
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    './data/test',# 학습셋이 있는 폴더의 위치
    target_size = (150, 150),
    batch_size = 5,
    class_mode = 'binary'
)
# CNN 모델을 만들어 적용
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

Found 160 images belonging to 2 classes.
Found 120 images belonging to 2 classes.




In [6]:
#모델 실행의 옵션을 설정
model.compile(loss="binary_crossentropy", \
              optimizer=optimizers.Adam(learning_rate=0.0002), metrics=['accuracy'])
# 학습의 조기 중단
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)
# 모델 학습
history = model.fit(train_generator, epochs=100, validation_data=test_generator, 
                  validation_steps= 10, callbacks=[early_stopping_callback])

Epoch 1/100


32/32 [==============================] - 4s 74ms/step - loss: 0.7070 - accuracy: 0.4125 - val_loss: 0.6675 - val_accuracy: 0.6400
Epoch 2/100
32/32 [==============================] - 1s 40ms/step - loss: 0.6870 - accuracy: 0.5688 - val_loss: 0.6778 - val_accuracy: 0.5600
Epoch 3/100
32/32 [==============================] - 1s 39ms/step - loss: 0.6922 - accuracy: 0.5188 - val_loss: 0.6795 - val_accuracy: 0.7800
Epoch 4/100
32/32 [==============================] - 1s 41ms/step - loss: 0.6877 - accuracy: 0.5312 - val_loss: 0.6763 - val_accuracy: 0.5200
Epoch 5/100
32/32 [==============================] - 1s 40ms/step - loss: 0.6805 - accuracy: 0.6250 - val_loss: 0.6759 - val_accuracy: 0.4600
Epoch 6/100
32/32 [==============================] - 1s 41ms/step - loss: 0.6501 - accuracy: 0.6500 - val_loss: 0.6589 - val_accuracy: 0.5000
Epoch 7/100
32/32 [==============================] - 1s 40ms/step - loss: 0.6389 - accuracy: 0.6438 - val_loss: 0.6227 - val_accuracy: 0.7200
Epoc

In [ ]:
# 전이 학습
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.layers import Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers, Input, models, layers,  metrics
from tensorflow.keras.applications import VGG16

# 내가 가진 이미지를 학습시키기
# 학습셋의 변형을 설정하는 부분
train_datagen = ImageDataGenerator(rescale=1./255,# 주어진 이미지의 크기를 설정
                                  horizontal_flip = True,#수평 대칭 이미지를 50% 확률로 만듦
                                  width_shift_range= 0.1, # 전체 크기의 15% 범위에서 좌우로 이동
                                  height_shift_range=0.1, # 전체 크기의 15% 범위에서위,아래로 이동
                                  #rotation_range = 5, #정해진 각도만큼 회전
                                  shear_range=0.7, #좌표 하나를 고정시키고 나머지를 이동시킴
                                  #zoom_range=1.2, # 확대 또는 축소
                                  #vertical_flip=True, # 수직 대칭 이미지를 만듦 
                                  fill_mode='nearest' # 빈 공간을 채우는 방법
                                   # nearest : 비슷한 색으로
                                  )
train_generator = train_datagen.flow_from_directory(
    './data/train',# 학습셋이 있는 폴더의 위치
    target_size = (150, 150),
    batch_size = 5,
    class_mode = 'binary'
)
# 테스트셋 설정
# 이미지 부풀리기 과정을 진행하지 않음 
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    './data/test',# 학습셋이 있는 폴더의 위치
    target_size = (150, 150),
    batch_size = 5,
    class_mode = 'binary'
)
# VGG16 모델
transfer_model = VGG16(weights='imagenet')